In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('m6_stocks_etfs_2010_updated.csv', index_col='Date', parse_dates=True)
#df.dropna(inplace= True)
df_ret = np.log(df/df.shift(1))

In [3]:
df_ret =df_ret[-160:].copy()  # 1 month data 
df_ret

,ABBV,ACN,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,...,XLF,XLV,XLE,XLY,XLI,XLC,XLU,XLP,XLB,VXX
Date,,,,,,,,,,,,,,,,,,,,,
2022-05-02,0.006717,0.014018,-0.009937,-0.005734,0.004716,0.023465,0.012428,0.001757,-0.028221,-0.007503,...,0.000290,-0.006236,0.015842,0.014506,0.002100,0.027225,-0.010297,-0.013225,-0.003063,-0.007607
2022-05-03,0.011632,0.000722,0.004575,0.012310,0.005387,0.007320,0.012128,-0.001982,0.005816,0.010654,...,0.012952,0.003162,0.027643,-0.002484,0.006586,0.002633,0.003821,-0.002875,0.011145,-0.016499
2022-05-04,0.013214,0.032407,0.021476,0.020490,0.026171,0.043083,0.041150,0.013390,0.000180,0.023134,...,0.030139,0.021930,0.040695,0.028937,0.028556,0.033770,0.022071,0.022263,0.031916,-0.084829
2022-05-05,0.003884,-0.052688,-0.010580,0.013606,-0.040484,-0.045987,-0.046271,-0.078622,-0.026231,-0.033676,...,-0.029567,-0.019623,-0.015160,-0.057675,-0.026474,-0.037557,-0.010276,-0.019127,-0.031450,0.082979
2022-05-06,0.004262,-0.009959,-0.000100,-0.004497,-0.010516,-0.008585,-0.010633,-0.014141,-0.019994,-0.010824,...,-0.007171,-0.003154,0.029371,-0.012754,-0.006782,-0.020836,0.007925,0.001043,-0.014446,0.002589
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-05,0.001709,-0.024931,-0.006328,0.008294,-0.031039,-0.002627,-0.027667,-0.033707,-0.013241,-0.006459,...,-0.025082,-0.009633,-0.030125,-0.029268,-0.016852,-0.015913,-0.005647,-0.012646,-0.020018,0.017236
2022-12-06,-0.001343,-0.025358,0.019384,-0.001854,-0.016804,-0.016293,0.001960,-0.030796,-0.020227,0.001939,...,-0.008886,-0.007613,-0.026714,-0.015714,-0.011897,-0.029177,0.006210,-0.006185,-0.008931,0.029632
2022-12-07,0.010209,0.004475,0.003108,-0.002013,-0.000183,0.013282,0.000621,0.002377,0.006450,0.004076,...,-0.004039,0.008191,-0.002347,-0.006038,-0.001308,-0.005048,-0.004654,0.004216,-0.002215,0.014493


In [4]:
df_ret.dropna(inplace= True)

In [5]:
df_ret.fillna(method='ffill', inplace= True)

### From historical data 

In [6]:
def get_percentile(df_ret):
    ret = df_ret.values.ravel()
    percentile = np.percentile(ret, [20,40,60,80])
    return percentile

def get_rank_probab(aapl, percentile):
    rank=np.zeros(5)
    rank[0] = len(np.where(aapl<percentile[0])[0])
    rank[1] = len(np.where(aapl<percentile[1])[0])-rank[0]
    rank[2]= len(np.where(aapl<percentile[2])[0])-rank[0]-rank[1]

    rank[3] = len(np.where(aapl<percentile[3])[0]) -rank[0] - rank[1]- rank[2]
    rank[4] = len(np.where(aapl>percentile[3])[0])
    
    return rank/rank.sum()

def get_ranking_csv(df_ret):
    percentile = get_percentile(df_ret)
    rank_proba_dict = {}
    for col in df_ret.columns:
        col_val= df_ret[col].values
        rank_proba_dict[col]=get_rank_probab(col_val, percentile)    
    df = pd.DataFrame(rank_proba_dict, index=range(1,6)).T
    return df

In [7]:
get_percentile(df_ret)

array([-0.0132037 , -0.00341872,  0.00337696,  0.01326868])

In [8]:
df_rank =  get_ranking_csv(df_ret)

In [9]:
df_rank

,1,2,3,4,5
ABBV,0.133858,0.236220,0.181102,0.291339,0.157480
ACN,0.228346,0.228346,0.173228,0.149606,0.220472
AEP,0.149606,0.181102,0.204724,0.299213,0.165354
AIZ,0.212598,0.157480,0.196850,0.251969,0.181102
ALLE,0.267717,0.141732,0.196850,0.133858,0.259843
...,...,...,...,...,...
XLC,0.259843,0.181102,0.196850,0.118110,0.244094
XLU,0.141732,0.220472,0.196850,0.307087,0.133858
XLP,0.102362,0.228346,0.299213,0.259843,0.110236
XLB,0.212598,0.196850,0.204724,0.188976,0.196850


In [10]:
import pandas as pd
from scipy import optimize 

def MaximizeSharpeRatioOptmzn(MeanReturns, CovarReturns, RiskFreeRate, PortfolioSize):
    
    # define maximization of Sharpe Ratio using principle of duality
    def  f(x, MeanReturns, CovarReturns, RiskFreeRate, PortfolioSize):
        funcDenomr = np.sqrt(np.matmul(np.matmul(x, CovarReturns), x.T) )
        funcNumer = np.matmul(np.array(MeanReturns),x.T)-RiskFreeRate
        func = -(funcNumer / funcDenomr)
        return func

    #define equality constraint representing fully invested portfolio
    def constraintEq(x):
        A=np.ones(x.shape)
        b=1
        constraintVal = np.matmul(A,x.T)-b 
        return constraintVal
    
    
    #define bounds and other parameters
    xinit=np.repeat(0.33, PortfolioSize)
    cons = ({'type': 'eq', 'fun':constraintEq})
    lb = 0
    ub = 1
    bnds = tuple([(lb,ub) for x in xinit])
    
    #invoke minimize solver
    opt = optimize.minimize (f, x0 = xinit, args = (MeanReturns, CovarReturns,\
                             RiskFreeRate, PortfolioSize), method = 'SLSQP',  \
                             bounds = bnds, constraints = cons, tol = 10**-3)
    
    return opt
    

In [11]:
arReturns = df_ret.values

#compute mean returns and variance covariance matrix of returns
meanReturns = np.mean(arReturns, axis = 0)
covReturns = np.cov(arReturns, rowvar=False)
# print('\nMean Returns:\n', meanReturns)
# print('\nVariance-Covariance Matrix of Returns:\n', covReturns)

In [12]:
df_ret.values.mean(axis=0).shape

(100,)

In [23]:
#obtain maximal Sharpe Ratio for k-portfolio 1 of Dow stocks

#set portfolio size
portfolioSize = 100


Rf=2.7
annRiskFreeRate = Rf/100


r0 = (np.power((1 + annRiskFreeRate),  (1.0 / 360.0)) - 1.0) * 100 
print('\nRisk free rate (daily %): ', end="")
print ("{0:.3f}".format(r0)) 

#initialization
xOptimal =[]
minRiskPoint = []
expPortfolioReturnPoint =[]
maxSharpeRatio = 0

#compute maximal Sharpe Ratio and optimal weights
result = MaximizeSharpeRatioOptmzn(meanReturns, covReturns, r0, portfolioSize)
xOptimal.append(result.x)

    
#compute risk returns and max Sharpe Ratio of the optimal portfolio   
xOptimalArray = np.array(xOptimal)
Risk = np.matmul((np.matmul(xOptimalArray,covReturns)), np.transpose(xOptimalArray))
expReturn = np.matmul(np.array(meanReturns),xOptimalArray.T)
annRisk =   np.sqrt(Risk*251) 
annRet = 251*np.array(expReturn) 
maxSharpeRatio = (annRet-Rf)/annRisk 

# #display results
# print('Maximal Sharpe Ratio: ', maxSharpeRatio, '\nAnnualized Risk (%):  ', \
#       annRisk, '\nAnnualized Expected Portfolio Return(%):  ', annRet)
# print('\nOptimal weights (%):\n',  xOptimalArray.T*100 )


Risk free rate (daily %): 0.007


In [24]:
df_weight = pd.DataFrame(np.round(xOptimalArray,2)[0], index= df_ret.columns, columns=['weight'])

In [25]:
# df_weight.loc[:,'weight'] = 0.0

In [26]:
# df_weight.loc['VXX', 'weight'] = 1.0

In [27]:
df_weight

,weight
ABBV,0.0
ACN,0.0
AEP,0.0
AIZ,0.0
ALLE,0.0
...,...
XLC,0.0
XLU,0.0
XLP,0.0
XLB,0.0


In [28]:
from portfolio_descision_module import *

In [29]:
final_csv = submit_csv(df_rank,df_weight)

In [32]:
final_csv.head(50)

,Rank1,Rank2,Rank3,Rank4,Rank5,Decision
ID,,,,,,
ABBV,0.13386,0.23622,0.18110,0.29134,0.15748,0.00
ACN,0.22835,0.22835,0.17322,0.14961,0.22047,0.00
AEP,0.14961,0.18110,0.20473,0.29921,0.16535,0.00
AIZ,0.21260,0.15748,0.19685,0.25197,0.18110,0.00
ALLE,0.26772,0.14173,0.19685,0.13386,0.25984,0.00
AMAT,0.33858,0.11811,0.10236,0.09449,0.34646,0.48
AMP,0.25197,0.14961,0.18110,0.12598,0.29134,0.00
AMZN,0.39370,0.10236,0.11812,0.10236,0.28346,0.00
AVB,0.22047,0.22835,0.17322,0.14961,0.22835,0.00


In [33]:
final_csv.to_csv('Submissions/Shrish_Dec_Submission.csv')